In [52]:
import pandas as pd
from sklearn.metrics import f1_score, auc, roc_curve
import numpy as np

In [53]:
# path = "/home/workspace/inference_result/CXR/llama_7b_openi_result_dabin.csv"
# path = "/home/workspace/inference_result/CXR/llama_13b_openi_result_dabin.csv"
# path = "/home/workspace/inference_result/CXR/llama-30b_openi_result_dabin.csv"
# path = "/home/workspace/inference_result/CXR/dabin_llama-65b-OPENI.csv"
# path = "/home/workspace/inference_result/CXR/dabin_llama-65b_NSHOT3-OPENI.csv"
# path = "/home/workspace/inference_result/CXR/dabin_llama-65b_NSHOT3-OPENI_4.csv"
# path = "/home/workspace/inference_result/CXR/dabin_llama-65b_NSHOT0-OPENI.csv"
path = "/home/workspace/inference_result/CXR/dabin_llama-65b_NSHOT0-OPENI_2.csv"

# path = "/home/workspace/inference_result/CXR/llama2_7b_openi_result_dabin.csv"
# path = "/home/workspace/inference_result/CXR/llama2_13b_openi_result_dabin.csv"
# path = "/home/workspace/inference_result/CXR/llama2-70b_openi_result_dabin.csv"

# path = "/home/workspace/inference_result/CXR/chexpert_openi_dabin.csv"
# path = "/home/workspace/inference_result/CXR/chexbert_openi_dabin.csv"

# path = "/home/workspace/inference_result/CXR/Llama-2-70b-hf_NSHOT3-OPENI_dabin.csv"
# path = "/home/workspace/inference_result/CXR/Llama-2-70b-hf_NSHOT10-OPENI_dabin.csv"

pred = pd.read_csv(path)

In [54]:
true = pd.read_csv('/home/data/CXR_Labeler/openi_500_answer_dabin.csv')
pred.columns =true.columns

In [55]:
drop_idx = []

#llama 7b
# drop_idx = [25, 199, 256, 278, 424, 489]

#llama 65b 3-shot
# drop_idx = [149]

#llama2 7b
# drop_idx = [385]

#llama2 70b 
# drop_idx = [107, 483, 494]

#Llama2 70b 3-shot
# drop_idx = [58, 107, 282, 429, 490]

In [56]:
pred = pred.drop(drop_idx)
true = true.drop(drop_idx).reset_index(drop=True)

In [57]:
pred = pred.astype('int64')

In [58]:
pred

,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,2,1,2,2,2,2,2,2,2,2,2,2,2
1,1,1,2,2,0,0,2,-1,0,1,2,2,2
2,0,0,0,0,0,0,0,2,0,0,0,0,0
3,2,1,2,2,2,2,2,2,0,0,2,2,2
4,0,0,0,0,0,0,0,-1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,2,2,2,2,2,2,2,2,2,2,2
496,0,0,2,2,2,2,2,2,0,0,2,0,0
497,2,2,2,2,2,2,2,2,2,2,2,2,2
498,0,0,2,2,2,2,2,2,0,0,2,0,0


In [59]:
true

,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,0,1,2,2,0,2,2,2,0,0,2,2,2
1,1,1,2,2,0,0,2,2,0,1,2,2,2
2,0,2,2,2,2,0,2,2,0,0,2,2,2
3,0,1,2,0,2,2,2,2,0,0,2,2,2
4,2,0,2,0,2,2,2,2,0,0,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,2,2,2,2,2,2,2,2,2,2,2
496,0,0,2,0,2,2,2,2,0,0,2,2,2
497,0,0,2,2,2,2,2,2,0,0,2,2,2
498,0,0,2,0,2,2,2,2,0,0,2,2,2


In [60]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from scipy import stats

def bootstrap_sample(df_pred, df_true, seed=None):
    np.random.seed(seed)
    n = len(df_pred)
    indices = np.random.choice(n, n, replace=True)
    new_pred = df_pred.iloc[indices].reset_index(drop=True)
    new_true = df_true.iloc[indices].reset_index(drop=True)
    return new_pred, new_true

def get_all_f1(true, pred, average):
    all_f1 = []
    for i in range(len(true.columns)):
        all_f1.append(f1_score(true.iloc[:,i], pred.iloc[:,i], average=average))
    return all_f1

all_f1_scores_weighted = []
all_f1_scores_macro = []
all_f1_scores_micro = []
for i in range(10):
    new_pred, new_true = bootstrap_sample(pred, true, seed=i)
    f1_weighted = get_all_f1(new_true, new_pred, average='weighted')
    f1_macro = get_all_f1(new_true, new_pred, average='macro')
    f1_micro = get_all_f1(new_true, new_pred, average='micro')
    all_f1_scores_weighted.append(f1_weighted)
    all_f1_scores_macro.append(f1_macro)
    all_f1_scores_micro.append(f1_micro)

def calculate_mean_std(scores_list):
    mean_scores = np.mean(scores_list, axis=0)
    std_scores = np.std(scores_list, axis=0)
    return mean_scores, std_scores

def calculate_confidence_interval(scores_list, confidence=0.95):
    mean_scores, std_scores = calculate_mean_std(scores_list)
    se = std_scores / np.sqrt(len(scores_list))
    h = se * stats.t.ppf((1 + confidence) / 2., len(scores_list) - 1)
    return mean_scores - h, mean_scores + h

# 평균, 표준편차 및 95% 신뢰구간 계산
average_f1_weighted, std_f1_weighted = calculate_mean_std(all_f1_scores_weighted)
ci_f1_weighted_1, ci_f1_weighted_2 = calculate_confidence_interval(all_f1_scores_weighted)

average_f1_macro, std_f1_macro = calculate_mean_std(all_f1_scores_macro)
ci_f1_macro_1, ci_f1_macro_2 = calculate_confidence_interval(all_f1_scores_macro)

average_f1_micro, std_f1_micro = calculate_mean_std(all_f1_scores_micro)
ci_f1_micro_1, ci_f1_micro_2 = calculate_confidence_interval(all_f1_scores_micro)

In [61]:
result_df = pd.DataFrame([average_f1_weighted, average_f1_macro, average_f1_micro]).transpose()
ci_df = pd.DataFrame([ci_f1_weighted_1, ci_f1_weighted_2, ci_f1_macro_1, ci_f1_macro_2, ci_f1_micro_1, ci_f1_micro_2]).transpose()

In [62]:
average_macro_f1 = [np.mean(f1) for f1 in all_f1_scores_macro]
pd.DataFrame(average_macro_f1)

,0
0,0.458063
1,0.463853
2,0.456194
3,0.468205
4,0.454066
5,0.448703
6,0.457858
7,0.434674
8,0.447890
9,0.452060


In [63]:
result_df

,0,1,2
0,0.514183,0.305564,0.5530
1,0.723203,0.527103,0.7382
2,0.719136,0.371601,0.5878
3,0.552298,0.434273,0.5398
4,0.719554,0.378908,0.6918
5,0.771580,0.443215,0.7572
6,0.711651,0.243501,0.5672
7,0.938575,0.556566,0.9024
8,0.782029,0.671319,0.7780
9,0.789321,0.740875,0.7718


In [64]:
ci_df

,0,1,2,3,4,5
0,0.500409,0.527958,0.263749,0.347379,0.538590,0.567410
1,0.708015,0.738391,0.516161,0.538045,0.726818,0.749582
2,0.704046,0.734226,0.341128,0.402073,0.568858,0.606742
3,0.535414,0.569182,0.403367,0.465179,0.522758,0.556842
4,0.705455,0.733653,0.347523,0.410294,0.676121,0.707479
5,0.757708,0.785453,0.419834,0.466596,0.742439,0.771961
6,0.697140,0.726163,0.201798,0.285205,0.549455,0.584945
7,0.931791,0.945359,0.530384,0.582748,0.892067,0.912733
8,0.770255,0.793802,0.596506,0.746133,0.765593,0.790407
9,0.779326,0.799316,0.714371,0.767378,0.761674,0.781926


In [65]:
average = [np.mean(all_f1_scores_macro[i]) for i in range(10)]
print(np.mean(average))
print(calculate_confidence_interval(average))

0.4541565188002411
(0.44784895899707633, 0.4604640786034059)


In [66]:
average = [np.mean(all_f1_scores_weighted[i]) for i in range(10)]
print(np.mean(average))
print(calculate_confidence_interval(average))

0.7041066927080486
(0.6985965587518911, 0.7096168266642061)
